In [2]:
import pandas as pd

df = pd.read_excel('Online Retail.xlsx')
b_matrix = pd.crosstab(df['InvoiceNo'], df['StockCode'])
b_matrix = (b_matrix > 0).astype(int)


step 3

In [3]:
import pandas as pd

df = pd.read_excel('Online Retail.xlsx')
df_sorted = df.sort_values('InvoiceDate')
batch_size = 10000
batches = [df_sorted.iloc[i:i + batch_size] for i in range(0, len(df_sorted), batch_size)]




step 4

In [7]:
import pandas as pd
import numpy as np
from sklearn.random_projection import GaussianRandomProjection
from sklearn.decomposition import IncrementalPCA

df = pd.read_excel('Online Retail.xlsx')
df_sorted = df.sort_values('InvoiceDate')
batch_size = 10000
batches = [df_sorted.iloc[i:i + batch_size] for i in range(0, len(df_sorted), batch_size)]

batch = batches[1]
for countr in range(1):
  #print(batch)
  binary_matrix = pd.crosstab(batch['InvoiceNo'], batch['StockCode'])
  binary_matrix = (binary_matrix > 0).astype(int)
  X = binary_matrix.to_numpy()


  rgp = GaussianRandomProjection(n_components=50)
  rgp_result = rgp.fit_transform(X)
  print(f"Random Gaussian Projection result shape: {rgp_result.shape}")

  ipca = IncrementalPCA(n_components=50)
  ipca.partial_fit(X)
  ipca_result = ipca.transform(X)
  print(f"Incremental PCA result shape: {ipca_result.shape}")



  l = 50
  n, d = X.shape
  fd = np.zeros((l, d))
  for i in range(n):
    a = X[i,:].reshape(1, -1)
    fd = np.vstack((fd, a))
    U, S, Vt = np.linalg.svd(fd, full_matrices=False)
    delta = S[l-1]**2
    S = np.sqrt(np.maximum(S**2 - delta, 0))
    fd = np.diag(S) @ Vt
    fd = fd[:l, :]
    ffd = X @ fd.T

  print(f"Frequesntly direction result shape: {ffd.shape}")




Random Gaussian Projection result shape: (427, 50)
Incremental PCA result shape: (427, 50)
Frequesntly direction result shape: (427, 50)


step 5

In [1]:
import pandas as pd
import numpy as np
from sklearn.random_projection import GaussianRandomProjection
from sklearn.decomposition import IncrementalPCA
from numpy.linalg import norm

df = pd.read_excel('Online Retail.xlsx')
df_sorted = df.sort_values('InvoiceDate')
batch_size = 10000
batches = [df_sorted.iloc[i:i + batch_size] for i in range(0, len(df_sorted), batch_size)]

#batch = batches[1]

for idx,batch in enumerate(batches):
  binary_matrix = pd.crosstab(batch['InvoiceNo'], batch['StockCode'])
  binary_matrix = (binary_matrix > 0).astype(int)
  X = binary_matrix.to_numpy()

  rgp = GaussianRandomProjection(n_components=50)
  rgp_result = rgp.fit_transform(X)
  W = rgp.components_
  re_rgp = rgp_result @ W
  print(f"Batch {idx+1}: Shape={rgp_result.shape}, Frobenius={norm(X - re_rgp, 'fro')}, MSE={np.mean((X - re_rgp) ** 2)}, ExplainedVar=Not Applicable")


  ipca = IncrementalPCA(n_components=50)
  ipca.partial_fit(X)
  ipca_result = ipca.transform(X)
  re_ipca = ipca.inverse_transform(ipca_result)
  explained_var = np.sum(ipca.explained_variance_ratio_)
  print(f"Batch {idx+1}: Shape={ipca_result.shape}, Frobenius={norm(X - re_ipca, 'fro')}, MSE={np.mean((X - re_ipca) ** 2)}, ExplainedVar={explained_var}")


  l = 50
  n, d = X.shape
  fd = np.zeros((l, d))
  for i in range(n):
    a = X[i,:].reshape(1, -1)
    fd = np.vstack((fd, a))
    U, S, Vt = np.linalg.svd(fd, full_matrices=False)
    delta = S[l-1]**2
    S = np.sqrt(np.maximum(S**2 - delta, 0))
    fd = np.diag(S) @ Vt
    fd = fd[:l, :]
    fd_result = X @ fd.T
  re_fd = fd_result @ np.linalg.pinv(fd)
  print(f"Batch {idx+1}: Shape={fd_result.shape}, Frobenius={norm(X - re_fd, 'fro')}, MSE={np.mean((X - re_fd) ** 2)}, ExplainedVar=Not Applicable")







FileNotFoundError: [Errno 2] No such file or directory: 'Online Retail.xlsx'